In [1]:
!git clone https://github.com/Tanveer2719/NIDS_Coursework.git

Cloning into 'NIDS_Coursework'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 147 (delta 58), reused 136 (delta 47), pack-reused 0 (from 0)
Receiving objects: 100% (147/147), 176.08 KiB | 3.26 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [2]:
%cd /kaggle/working/NIDS_Coursework/My_Code
!ls

/kaggle/working/NIDS_Coursework/My_Code
classification.py    main.ipynb     record_level_embedding.py
featureSelection.py  preprocess.py  transformer.py


In [3]:
%rm -r __pycache__

rm: cannot remove '__pycache__': No such file or directory


In [4]:
!git pull origin main

From https://github.com/Tanveer2719/NIDS_Coursework
 * branch            main       -> FETCH_HEAD
Already up to date.


In [5]:
import pandas as pd
import numpy as np

path = "/kaggle/input/unsw-nb-15/UNSW-NB15_combined.csv"

df = pd.read_csv(path)

print(df.shape)
df = df.drop(columns=['id', 'label'])
print(df.shape)

print(f"🧾 Unique class labels: {df['attack_cat'].unique()}")
df['label'] = df['attack_cat'].apply(lambda x: 0 if x == 'Normal' else 1)

print(f'Unique values for label {df["label"].unique()}')

df = df.drop(columns=["attack_cat"])
print(f'new shape{df.shape}')

print(df.dtypes)


(257673, 45)
(257673, 43)
🧾 Unique class labels: ['Normal' 'Backdoor' 'Analysis' 'Fuzzers' 'Shellcode' 'Reconnaissance'
 'Exploits' 'DoS' 'Worms' 'Generic']
Unique values for label [0 1]
new shape(257673, 43)
dur                  float64
proto                 object
service               object
state                 object
spkts                  int64
dpkts                  int64
sbytes                 int64
dbytes                 int64
rate                 float64
sttl                   int64
dttl                   int64
sload                float64
dload                float64
sloss                  int64
dloss                  int64
sinpkt               float64
dinpkt               float64
sjit                 float64
djit                 float64
swin                   int64
stcpb                  int64
dtcpb                  int64
dwin                   int64
tcprtt               float64
synack               float64
ackdat               float64
smean                  int64
dmean   

In [6]:
# perform preprocessing

from preprocess import Preprocess

# separate the features and labels so that the labesl are not encoded
labels = df['label']
features = df.drop(columns=['label'])

pp = Preprocess()
processed_features = pp.fit_transform_df_auto(df = features,n_categorical_levels=32, expected_categorical_format='onehot')

print(processed_features.shape)
processed_features.head()

Encoding the 32 levels for proto
Encoding the 13 levels for service
Encoding the 11 levels for state
(257673, 95)


,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,state_2,state_3,state_4,state_5,state_6,state_7,state_8,state_9,state_10,state_11
0,0.027889,0.193225,0.172921,0.331292,0.312312,0.312595,0.997874,1.000000,0.424562,0.534539,...,False,False,False,False,False,False,False,False,False,False
1,0.121803,0.284598,0.393619,0.398471,0.645181,0.316704,0.747160,0.998579,0.401347,0.775718,...,False,False,False,False,False,False,False,False,False,False
2,0.234590,0.224248,0.304405,0.353884,0.574953,0.196832,0.747160,0.998579,0.326962,0.650937,...,False,False,False,False,False,False,False,False,False,False
3,0.239956,0.267974,0.275582,0.388675,0.402879,0.194440,0.747160,0.998579,0.351625,0.479722,...,False,False,False,False,False,False,False,False,False,False
4,0.090294,0.248312,0.209072,0.378428,0.339064,0.256038,0.999294,0.998579,0.402218,0.489854,...,False,False,False,False,False,False,False,False,False,False


In [7]:
# concat the features and the labels
processed_full_df = pd.concat([processed_features, labels], axis=1)
processed_full_df.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,state_3,state_4,state_5,state_6,state_7,state_8,state_9,state_10,state_11,label
0,0.027889,0.193225,0.172921,0.331292,0.312312,0.312595,0.997874,1.000000,0.424562,0.534539,...,False,False,False,False,False,False,False,False,False,0
1,0.121803,0.284598,0.393619,0.398471,0.645181,0.316704,0.747160,0.998579,0.401347,0.775718,...,False,False,False,False,False,False,False,False,False,0
2,0.234590,0.224248,0.304405,0.353884,0.574953,0.196832,0.747160,0.998579,0.326962,0.650937,...,False,False,False,False,False,False,False,False,False,0
3,0.239956,0.267974,0.275582,0.388675,0.402879,0.194440,0.747160,0.998579,0.351625,0.479722,...,False,False,False,False,False,False,False,False,False,0
4,0.090294,0.248312,0.209072,0.378428,0.339064,0.256038,0.999294,0.998579,0.402218,0.489854,...,False,False,False,False,False,False,False,False,False,0


In [8]:
from featureSelection import KBestFeatureSelector

kbest = KBestFeatureSelector(k=65)  # Set k to your desired number of features
selected_mask, selected_feature = kbest.select_features(df=processed_full_df, target_column='label', verbose=True)

selected_feature

2025-07-19 05:32:01.203461: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752903121.376679      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752903121.424833      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



🔍 Starting SelectKBest feature selection (k = 65) using mutual_info_classif...
✅ Selected top 65 features out of 95
📌 Selected features:
   1. dur
   2. spkts
   3. dpkts
   4. sbytes
   5. dbytes
   6. rate
   7. sttl
   8. dttl
   9. sload
  10. dload
  11. sloss
  12. dloss
  13. sinpkt
  14. dinpkt
  15. sjit
  16. djit
  17. swin
  18. stcpb
  19. dtcpb
  20. dwin
  21. tcprtt
  22. synack
  23. ackdat
  24. smean
  25. dmean
  26. trans_depth
  27. response_body_len
  28. ct_srv_src
  29. ct_state_ttl
  30. ct_dst_ltm
  31. ct_src_dport_ltm
  32. ct_dst_sport_ltm
  33. ct_dst_src_ltm
  34. ct_flw_http_mthd
  35. ct_src_ltm
  36. ct_srv_dst
  37. is_sm_ips_ports
  38. proto_1
  39. proto_2
  40. proto_3
  41. proto_4
  42. proto_5
  43. proto_6
  44. proto_7
  45. proto_8
  46. proto_9
  47. proto_13
  48. proto_18
  49. proto_19
  50. proto_27
  51. proto_28
  52. proto_29
  53. proto_31
  54. service_1
  55. service_2
  56. service_3
  57. service_5
  58. service_7
  59. servic

Index(['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl',
       'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit',
       'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean',
       'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src',
       'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
       'ct_dst_src_ltm', 'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst',
       'is_sm_ips_ports', 'proto_1', 'proto_2', 'proto_3', 'proto_4',
       'proto_5', 'proto_6', 'proto_7', 'proto_8', 'proto_9', 'proto_13',
       'proto_18', 'proto_19', 'proto_27', 'proto_28', 'proto_29', 'proto_31',
       'service_1', 'service_2', 'service_3', 'service_5', 'service_7',
       'service_8', 'service_11', 'service_13', 'state_1', 'state_2',
       'state_3', 'state_4'],
      dtype='object')

In [9]:
columns_to_keep = selected_feature

In [10]:
df_selected = processed_full_df[columns_to_keep].copy()
df_selected.shape

(257673, 65)

In [11]:
df_selected.dtypes

dur           float64
spkts         float64
dpkts         float64
sbytes        float64
dbytes        float64
               ...   
service_13       bool
state_1          bool
state_2          bool
state_3          bool
state_4          bool
Length: 65, dtype: object

In [12]:
numerical_columns = [col for col in columns_to_keep 
                     if not (col.startswith("proto_") or 
                             col.startswith("service_") or 
                             col.startswith("state_"))]


In [13]:
from record_level_embedding import RecordLevelEmbedder

embedder = RecordLevelEmbedder(selected_df = df_selected, numerical_columns = numerical_columns, embed_dimension= 64)

embedded_df = embedder.transform_to_df()

embedded_df.head()

[INFO] Initializing RecordLevelEmbedder...
[INFO] Detecting categorical blocks...
[INFO] Detected 3 categorical blocks:
   - proto: 16 columns
   - service: 8 columns
   - state: 4 columns
[INFO] Embedder initialized with embedding dimension 64.
[INFO] Building Keras embedding model...
   - Adding numerical input: dur
   - Adding numerical input: spkts
   - Adding numerical input: dpkts
   - Adding numerical input: sbytes
   - Adding numerical input: dbytes
   - Adding numerical input: rate
   - Adding numerical input: sttl
   - Adding numerical input: dttl
   - Adding numerical input: sload
   - Adding numerical input: dload
   - Adding numerical input: sloss
   - Adding numerical input: dloss
   - Adding numerical input: sinpkt
   - Adding numerical input: dinpkt
   - Adding numerical input: sjit
   - Adding numerical input: djit
   - Adding numerical input: swin
   - Adding numerical input: stcpb
   - Adding numerical input: dtcpb
   - Adding numerical input: dwin
   - Adding numeri

I0000 00:00:1752903265.043195      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1752903265.043843      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


[INFO] Projection complete. Output embedding shape: (64,)
[INFO] Model build complete.
[INFO] Preparing inputs for embedding. Number of rows: 257673
   - Prepared numerical input: dur
   - Prepared numerical input: spkts
   - Prepared numerical input: dpkts
   - Prepared numerical input: sbytes
   - Prepared numerical input: dbytes
   - Prepared numerical input: rate
   - Prepared numerical input: sttl
   - Prepared numerical input: dttl
   - Prepared numerical input: sload
   - Prepared numerical input: dload
   - Prepared numerical input: sloss
   - Prepared numerical input: dloss
   - Prepared numerical input: sinpkt
   - Prepared numerical input: dinpkt
   - Prepared numerical input: sjit
   - Prepared numerical input: djit
   - Prepared numerical input: swin
   - Prepared numerical input: stcpb
   - Prepared numerical input: dtcpb
   - Prepared numerical input: dwin
   - Prepared numerical input: tcprtt
   - Prepared numerical input: synack
   - Prepared numerical input: ackdat
  

I0000 00:00:1752903266.729449     118 service.cc:148] XLA service 0x7e5058012670 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752903266.730201     118 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1752903266.730241     118 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1752903266.763141     118 cuda_dnn.cc:529] Loaded cuDNN version 90300


  53/8053 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step    

I0000 00:00:1752903267.060696     118 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


8053/8053 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step
[INFO] Embedding complete. Output shape: (257673, 64)
[INFO] Embedding DataFrame ready.


,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_54,embed_55,embed_56,embed_57,embed_58,embed_59,embed_60,embed_61,embed_62,embed_63
0,0.784787,0.104468,-0.300666,0.280363,-0.204538,-0.082034,0.121907,0.543349,-0.666857,-0.362320,...,0.677513,0.048563,-0.379054,0.232934,-0.097277,-0.093644,0.386860,0.275907,0.381084,0.268424
1,1.103540,0.311832,-0.222792,0.384229,-0.336493,0.073478,0.217422,0.749644,-0.494502,-0.251600,...,0.711536,-0.062710,-0.154942,0.414993,-0.211940,0.009553,0.299005,0.530748,0.183580,0.558750
2,0.982838,0.247653,-0.379718,0.431306,-0.377890,0.057269,0.211256,0.725221,-0.613003,-0.371039,...,0.697004,0.098058,-0.212068,0.312164,-0.292553,-0.087927,0.376697,0.528844,0.260505,0.392304
3,0.770256,0.056126,-0.560612,0.128857,-0.328612,0.103758,0.206279,0.505521,-0.805545,-0.304209,...,0.530082,0.092287,-0.285762,0.173820,-0.044190,-0.080289,0.586616,0.376686,0.278377,0.325165
4,1.038314,0.211607,-0.347283,0.635245,-0.224253,0.186243,0.424433,0.834891,-0.564898,-0.488774,...,0.700587,0.070475,-0.294788,0.254107,-0.216200,-0.245935,-0.024432,0.643957,0.145546,0.398081


In [14]:
# Binary classification
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model


print(processed_full_df['label'].unique())

binary_target = (processed_full_df['label'] != 0).astype(int).values
unique, counts = np.unique(binary_target, return_counts=True)
print("Classes and their counts:")
for cls, cnt in zip(unique, counts):
    print(f"Class {cls}: {cnt}")

[0 1]
Classes and their counts:
Class 0: 93000
Class 1: 164673


In [15]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from transformer import BasicTransformer
from classification import LastTokenClassificationHead
from sklearn.model_selection import train_test_split
import numpy as np  # Just in case

# Input data
X_embed = embedded_df.values                   # shape: (num_samples, embed_dim)
X_embed_seq = tf.expand_dims(X_embed, axis=1)  # shape: (batch_size, 1, embed_dim)

input_layer = Input(shape=(1, X_embed.shape[1]), name="record_input")

# Transformer block
transformer = BasicTransformer(
    n_layers=2,
    internal_size=128,
    n_heads=4,
    verbose=True  # See progress!
)

x = transformer.apply(input_layer, training=True)

# Classification head
classification_head = LastTokenClassificationHead()
x = classification_head.apply(x)

# MLP head
for i, layer_size in enumerate([128]):
    x = Dense(layer_size, activation="relu", name=f"classification_mlp_{i}_{layer_size}")(x)
    x = Dropout(0.1)(x)

# Output layer for binary classification
output_layer = Dense(1, activation="sigmoid", name="binary_output")(x)

# Build the model
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

# Compile the model
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
    jit_compile=True
)

# Prepare data
X_np = X_embed_seq.numpy() if hasattr(X_embed_seq, 'numpy') else np.array(X_embed_seq)

X_train, X_val, y_train, y_val = train_test_split(
    X_np, binary_target, test_size=0.2, random_state=42, stratify=binary_target
)

# Define early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Train the model
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop]
)


[BasicTransformer] Applying Encoder Block 1/2
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[BasicTransformer] Applying Encoder Block 2/2
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[BasicTransformer] Final output shape: (None, 1, 64)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ record_input (InputLayer)       │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_0_transformer_encoder     │ (None, 1, 64)          │       149,504 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_1_transformer_encoder     │ (None, 1, 64)          │       149,504 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ slice_last (Lambda)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classification_mlp_0_128        │ (None, 128)            │         8,320 │
│ (Dense)                         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ binary_output (Dense)           │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 307,457 (1.17 MB)

 Trainable params: 307,457 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9010 - loss: 0.1974[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 45s 5ms/step - accuracy: 0.9010 - loss: 0.1973 - val_accuracy: 0.9208 - val_loss: 0.1613
Epoch 2/50
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.9167 - loss: 0.1670 - val_accuracy: 0.9246 - val_loss: 0.1514
Epoch 3/50
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.9236 - loss: 0.1544 - val_accuracy: 0.9271 - val_loss: 0.1523
Epoch 4/50
6442/6442 

In [16]:
# evaluate the model
results = model.evaluate(X_val, y_val)
print(f"Validation loss: {results[0]:.4f}, Validation accuracy: {results[1]:.4f}")


1611/1611 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9380 - loss: 0.1251
Validation loss: 0.1262, Validation accuracy: 0.9379


In [17]:
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np

# Predict probabilities
y_pred_prob = model.predict(X_val)
y_pred = (y_pred_prob >= 0.5).astype(int).flatten()

# F1
f1 = f1_score(y_val, y_pred)
print(f"F1 Score: {f1:.4f}")

# Confusion matrix
cm = confusion_matrix(y_val, y_pred, labels=[0, 1])
print("Confusion Matrix:\n", cm)

if cm.shape == (2, 2):
    tn, fp, fn, tp = cm.ravel()
    detection_rate = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    print(f"Detection Rate (Recall): {detection_rate:.4f}")
    print(f"False Alarm Rate: {false_alarm_rate:.4f}")
else:
    print("Confusion matrix does not contain both classes!")
    print("Check your labels: y_val:", np.unique(y_val), " y_pred:", np.unique(y_pred))


[EncoderBlock] Input shape: (32, 1, 64)
[EncoderBlock] Output shape: (32, 1, 64)
[EncoderBlock] Input shape: (32, 1, 64)
[EncoderBlock] Output shape: (32, 1, 64)
1609/1611 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
1611/1611 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
F1 Score: 0.9507
Confusion Matrix:
 [[17466  1134]
 [ 2066 30869]]
Detection Rate (Recall): 0.9373
False Alarm Rate: 0.0610
